<a href="https://colab.research.google.com/github/Chaithanya3K/smart-food-waste-detection/blob/main/gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import gradio as gr
import torch
import time
import tensorflow as tf
from PIL import Image
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation

# ==========================
# 1. LOAD MODELS (HYBRID LOGIC)
# ==========================
# Ensure unet_model.h5 is in your directory
unet_model = tf.keras.models.load_model("/content/drive/MyDrive/unet_food_2_segmentation.h5")

# SegFormer for ADE20K (Class 13 = Food)
processor = SegformerImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
vit_model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
vit_model.eval()

# ==========================
# 2. HYBRID PROCESSING LOGIC
# ==========================
def get_hybrid_food_mask(img_rgb):
    # --- U-Net Prediction ---
    unet_size = (256, 256)
    img_unet = cv2.resize(img_rgb, unet_size)
    unet_input = np.expand_dims(img_unet / 255.0, axis=0)
    unet_pred = unet_model.predict(unet_input, verbose=0)[0]
    if len(unet_pred.shape) == 3:
        unet_pred = unet_pred[:, :, 0]
    unet_mask = (unet_pred > 0.5).astype(np.uint8)

    # --- SegFormer Prediction ---
    inputs = processor(images=img_rgb, return_tensors="pt")
    with torch.no_grad():
        outputs = vit_model(**inputs)

    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(
        logits, size=unet_size, mode='bilinear', align_corners=False
    )
    vit_labels = torch.argmax(upsampled_logits, dim=1).squeeze().cpu().numpy()
    vit_mask = (vit_labels == 13).astype(np.uint8)

    # --- Combine (OR Logic) & Refine ---
    combined = cv2.bitwise_or(unet_mask, vit_mask)
    kernel = np.ones((5, 5), np.uint8)
    refined = cv2.morphologyEx(combined, cv2.MORPH_CLOSE, kernel)
    refined = cv2.morphologyEx(refined, cv2.MORPH_OPEN, kernel)

    return refined

# ==========================
# 3. UI HTML COMPONENTS (EXACT STYLE)
# ==========================
def gauge_html(pct):
    return f"""
    <div class="visual-gauge-container">
        <div class="gauge-header">Food wasted %</div>
        <div class="gauge-track-wrapper">
            <div class="gauge-track">
                <div class="gauge-cursor" style="left: {pct}%;"></div>
            </div>
            <div class="gauge-labels-container">
                <span style="left: 0%;">0%</span>
                <span style="left: 25%;">25%</span>
                <span style="left: 50%;">50%</span>
                <span style="left: 100%;">100%</span>
            </div>
        </div>
    </div>
    """

def alert_html(remaining):
    if remaining > 30:
        badge = "High" if remaining > 50 else "Medium"
        color = "#e67e22"
        return f"""
        <div class="custom-alert-card">
            <div class="alert-header">
                <span>FOOD WASTE ALERT & TIPS</span>
                <span class="alert-badge" style="background:{color}">{badge}</span>
            </div>
            <p>Someone, somewhere, is sleeping hungry tonight while food lies wasted here.</p>
            <p><b>Food is not just a meal</b> — it is time, effort, and nature’s gift.</p>
            <ul>
                <li> Please take only what you need</li>
                <li> Respect food. Reduce waste</li>
            </ul>
        </div>
        """
    else:
        return f"""
        <div class="custom-alert-card">
            <div class="alert-header">
                <span>THANK YOU FOR BEING RESPONSIBLE </span>
                <span class="alert-badge" style="background:#27ae60">Low</span>
            </div>
            <p>You avoided food waste and showed mindfulness. Every small decision matters.</p>
        </div>
        """

# ==========================
# 4. MAIN PIPELINE
# ==========================
def food_waste_pipeline(before_img, after_img):
    if before_img is None or after_img is None: return [None]*10

    start = time.time()

    # Process Images
    img_b = np.array(before_img)
    img_a = np.array(after_img)

    mask_b = get_hybrid_food_mask(img_b)
    mask_a = get_hybrid_food_mask(img_a)

    count_b = np.sum(mask_b == 1)
    count_a = np.sum(mask_a == 1)

    # Calculate Waste (Using your 0.8 correction factor)
    remaining = 0.0
    if count_b > 0:
        corrected_after = count_a * 0.8
        remaining = min((corrected_after / count_b) * 100, 100.0)

    eaten = 100 - remaining
    waste_g = round(remaining * 5.2, 1)  # Est weight
    co2_g = round(waste_g * 2.0, 1)      # Est CO2

    # UI Cards
    m1 = f'<div class="m-card"><span>Waste %</span><h3>{remaining:.1f}%</h3><div class="m-bar"><div class="m-fill red" style="width:{remaining}%"></div></div></div>'
    m2 = f'<div class="m-card"><span>Eaten %</span><h3>{eaten:.1f}%</h3><div class="m-bar"><div class="m-fill green" style="width:{eaten}%"></div></div></div>'
    m3 = f'<div class="m-card"><span>Waste (g)</span><h3>{waste_g}g</h3><div class="m-bar"><div class="m-fill orange" style="width:{min(100, waste_g/5)}%"></div></div></div>'
    m4 = f'<div class="m-card"><span>CO₂ impact (g)</span><h3>{co2_g}g</h3><div class="m-bar"><div class="m-fill lightgreen" style="width:{min(100, co2_g/10)}%"></div></div></div>'

    # Convert masks for display
    pat_b = (mask_b * 255).astype(np.uint8)
    pat_a = (mask_a * 255).astype(np.uint8)

    status = f'<div class="processing-bar"><span>✅ Complete</span><span>{round(time.time()-start,2)}s</span></div>'

    return (m1, m2, m3, m4, gauge_html(remaining), alert_html(remaining), status, pat_b, pat_a, gr.update(visible=True))

# ==========================
# 5. CSS (EXACT STYLE)
# ==========================
custom_css = """
:root { --bg-color: #111827; --card-bg: #1f2937; --text-main: #ffffff; --text-sub: #9ca3af; --img-border: #374151; }
body, .gradio-container { background-color: var(--bg-color) !important; }
.calc-btn { background-color: #e67e22 !important; color: white !important; border-radius: 12px !important; font-weight: bold !important; }
.reset-btn { background-color: transparent !important; color: #9ca3af !important; border-radius: 12px !important; border: 1px solid var(--img-border) !important; }
.visual-gauge-container { margin: 20px auto; max-width: 90%; }
.gauge-track { height: 14px; border-radius: 7px; background: linear-gradient(90deg, #4ade80 0%, #facc15 50%, #ef4444 100%); position: relative; }
.gauge-cursor { position: absolute; top: -6px; height: 26px; width: 4px; background: #ffffff; box-shadow: 0 0 10px white; transform: translateX(-50%); transition: left 0.8s ease; }
.gauge-labels-container { position: relative; width: 100%; height: 20px; margin-top: 10px; color: var(--text-sub); font-size: 12px; }
.gauge-labels-container span { position: absolute; transform: translateX(-50%); }
.custom-alert-card { background: var(--card-bg); border-radius: 15px; padding: 20px; border: 1px solid var(--img-border); color: var(--text-main); margin-top: 20px; }
.alert-header { display: flex; justify-content: space-between; align-items: center; margin-bottom: 15px; font-weight: bold; }
.alert-badge { padding: 4px 12px; border-radius: 20px; font-size: 12px; color: white; }
.m-card { background: var(--card-bg); border-radius: 16px; padding: 18px 20px; color: var(--text-main); border: 1px solid var(--img-border); }
.m-bar { height: 6px; background: rgba(156, 163, 175, 0.2); border-radius: 6px; margin-top: 10px; }
.m-fill { height:100%; border-radius:6px; }
.red { background: #ef4444; } .green { background: #22c55e; } .orange { background: #f59e0b; } .lightgreen { background: #84cc16; }
.img-container { background: var(--card-bg); border-radius: 12px; padding: 15px; border: 1px solid var(--img-border); }
.processing-bar { display:flex; justify-content:space-between; background: var(--card-bg); color: var(--text-main); padding:8px 12px; border-radius:12px; border: 1px solid var(--img-border); }
"""

# ==========================
# 6. UI CONSTRUCTION
# ==========================
with gr.Blocks(css=custom_css) as demo:
    gr.HTML('<div style="background:var(--card-bg); padding:20px; border-radius:15px; text-align:center; color:white; border:1px solid var(--img-border); margin-top:20px;"><h2>Hybrid Food Waste Analysis</h2><p>U-Net + Vision Transformer (SegFormer) Integration</p></div>')

    with gr.Row():
        with gr.Column(elem_classes="img-container"):
            gr.HTML('<div style="color:white; font-weight:bold; margin-bottom:10px;">Before Meal</div>')
            before_input = gr.Image(show_label=False, type="pil")
        with gr.Column(elem_classes="img-container"):
            gr.HTML('<div style="color:white; font-weight:bold; margin-bottom:10px;">After Meal</div>')
            after_input = gr.Image(show_label=False, type="pil")

    with gr.Row():
        submit_btn = gr.Button("Calculate Food Waste", elem_classes="calc-btn", scale=3)
        reset_btn  = gr.Button("Reset", elem_classes="reset-btn", scale=1)

    status_area = gr.HTML('<div class="processing-bar"><span>⏳ Waiting</span><span>—</span></div>')

    with gr.Column(visible=False) as results_section:
        gauge_out = gr.HTML()
        with gr.Row():
            m1_out = gr.HTML(); m2_out = gr.HTML(); m3_out = gr.HTML(); m4_out = gr.HTML()
        alert_area = gr.HTML()

        with gr.Accordion("🔍 View Hybrid Segmentation Patterns", open=False):
            with gr.Row():
                pat_before = gr.Image(label="Before Mask", height=300)
                pat_after = gr.Image(label="After Mask", height=300)

    submit_btn.click(
        food_waste_pipeline,
        inputs=[before_input, after_input],
        outputs=[m1_out, m2_out, m3_out, m4_out, gauge_out, alert_area, status_area, pat_before, pat_after, results_section]
    )

    reset_btn.click(
        lambda: (None, None, gr.update(visible=False), '<div class="processing-bar"><span>⏳ Waiting</span><span>—</span></div>'),
        outputs=[before_input, after_input, results_section, status_area]
    )

if __name__ == "__main__":
    demo.launch(debug=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/image_processing_base.py:417: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type', 'reduce_labels'
  image_processor = cls(**image_processor_dict)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/15.0M [00:00<?, ?B/s]

/tmp/ipython-input-3472835560.py:173: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fc35930b5a6053d06b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fc35930b5a6053d06b.gradio.live
